In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import flirt.reader.empatica
import flirt.with_
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
from tqdm.autonotebook import trange
from datetime import timedelta
import multiprocessing

c:\Users\duong\anaconda3\envs\batf\lib\site-packages\flirt\acc\feature_calculation.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange
c:\Users\duong\anaconda3\envs\batf\lib\site-packages\flirt\lib\entropy\fractal.py:106: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit('float64(float64[:], int32)')


Import Datasets

In [2]:
raw_acceleration_data = pd.read_csv("C:/Users/duong/Desktop/BA/Analysis/dataset-main/AccelerometerMeasurements.csv")
raw_selfreports_data = pd.read_csv("C:/Users/duong/Desktop/BA/Analysis/dataset-main/SelfReports.csv")

Copy Dataset

In [3]:
df_accel = raw_acceleration_data.copy()
df_reports = raw_selfreports_data.copy()

Cleaning Reports (Removing not available data)

In [4]:
df_reports = df_reports[df_reports.timeOfEngagement != 0].copy()

df_reports = df_reports[df_reports.valence != "NONE"].copy()
df_reports = df_reports[df_reports.arousal != "NONE"].copy()
df_reports = df_reports[df_reports.context != "NONE"].copy()


Renaming Column in df Accel

In [5]:
df_accel.rename(columns={'timestamp':'timeOfNotification'}, inplace=True)

Convert Timenotification form Unix to DateTime

In [6]:
df_accel["timeOfNotification"] = pd.to_datetime(df_accel["timeOfNotification"], unit="ms")
df_reports["timeOfNotification"] = pd.to_datetime(df_reports["timeOfNotification"], unit="ms")


Function to extract the Timewindow

In [7]:
def extract_accel_data(row, accel_data):
    start_time = row['timeOfNotification'] - pd.Timedelta(milliseconds=300000)
    end_time = row['timeOfNotification'] + pd.Timedelta(milliseconds=300000)
    participant_id = row['participantId']
    
    # Filter accelerometer data for the given participant and time window
    mask = (
        (accel_data['participantId'] == participant_id) &
        (accel_data['timeOfNotification'] >= start_time) &
        (accel_data['timeOfNotification'] <= end_time)
    )
    return accel_data[mask]

In [8]:
df_reports['accel_data'] = df_reports.apply(extract_accel_data, accel_data=df_accel, axis=1)

Create Combined Dataframe

In [9]:
combined_data = []

for _, row in df_reports.iterrows():
    accel_data = row['accel_data']
    for _, accel_row in accel_data.iterrows():
        combined_data.append({
            'participantId': row['participantId'],
            'selfreport_time': row['timeOfNotification'],
            'valence': row['valence'],
            'arousal': row['arousal'],
            'context': row['context'],
            'accel_time': accel_row['timeOfNotification'],
            'x': accel_row['x'],
            'y': accel_row['y'],
            'z': accel_row['z']
        })

combined_df = pd.DataFrame(combined_data)

In [10]:
combined_df.to_csv("data_5min_ms", encoding='utf-8', index=False)

Import combined Dataset

In [12]:
data = pd.read_csv("data_5min_ms.csv")

Label with ReportID

In [13]:
data['reportId'] = data.groupby(['participantId', 'selfreport_time']).ngroup() + 1

In [14]:
col = data.pop("reportId")
data.insert(0, col.name, col)

Scale x y z data

In [15]:
columns_to_scale = ['x', 'y', 'z']
scaler = StandardScaler()
scaled_columns = scaler.fit_transform(data[columns_to_scale])
scaled_df = pd.DataFrame(scaled_columns, columns=columns_to_scale, index=data.index)
data[columns_to_scale] = scaled_df

In [16]:
data = data.drop('participantId', axis=1)
data = data.drop('selfreport_time', axis=1)

In [17]:
# FUNKTIONIERT OHNE INPLACE NICHT (weil inplace den originalen df modifiziert, sonst nicht)
data.rename(columns={"accel_time": "datetime"}, inplace=True)

In [18]:
data.shape

(4869564, 8)

In [19]:
data['datetime'] = pd.DatetimeIndex(data["datetime"]).astype( np.int64 )/100000
# combined_df['accel_time'] = pd.DatetimeIndex(combined_df["accel_time"]).astype( np.int64 )/100000

In [20]:
data.reportId

0            1
1            1
2            1
3            1
4            1
          ... 
4869559    330
4869560    330
4869561    330
4869562    330
4869563    330
Name: reportId, Length: 4869564, dtype: int64

Test first five

In [21]:
# features_list = []

# # Iterate through each group
# for report_id in data['reportId'].unique()[:3]:

#     # Creating temporaire dataframe with rows with same reportId
#     temp = data[data['reportId'] == report_id]

#     # Extracting accel_time x y z
#     temp_ex = temp[['datetime',"x","y","z"]].copy()

#     # Getting features
#     features = flirt.get_acc_features(temp_ex, window_length=10, window_step_size=1, data_frequency=25)

#     # Adding reportId, arousal, valence, and context to every row
#     features['reportId'] = report_id
#     features['arousal'] = temp['arousal'].iloc[0]
#     features['valence'] = temp['valence'].iloc[0]
#     features['context'] = temp['context'].iloc[0]
    
#     # Append to the list
#     features_list.append(features)


# # Combine all feature DataFrames into a single DataFrame
# all_features = pd.concat(features_list)

In [22]:
# all_features.shape

In [23]:
# all_features.to_csv("allfeatures_3min_10_1", encoding='utf-8', index=False)

Analyze Data and put into list

In [24]:
features_list = []

# Iterate through each group
for report_id in data['reportId'].unique():

    # Creating temporaire dataframe with rows with same reportId
    temp = data[data['reportId'] == report_id]

    # Extracting accel_time x y z
    temp_ex = temp[['datetime',"x","y","z"]].copy()

    # Getting features
    features = flirt.get_acc_features(temp_ex, window_length=120, window_step_size=10, data_frequency=25)

    # Adding reportId, arousal, valence, and context to every row
    features['reportId'] = report_id
    features['arousal'] = temp['arousal'].iloc[0]
    features['valence'] = temp['valence'].iloc[0]
    features['context'] = temp['context'].iloc[0]
    
    # Append to the list
    features_list.append(features)


# Combine all feature DataFrames into a single DataFrame
all_features = pd.concat(features_list)

ACC features:   0%|          | 0/60 [00:00<?, ?it/s]

ACC features: 100%|██████████| 60/60 [00:26<00:00,  2.27it/s]


Unable to remove memmapped file


ACC features: 100%|██████████| 60/60 [00:00<00:00, 445.35it/s]


Unable to remove memmapped file


...

Unable to remove memmapped file


ACC features: 100%|██████████| 60/60 [00:00<00:00, 487.80it/s]


Unable to remove memmapped file


ACC features: 100%|██████████| 60/60 [00:00<00:00, 398.03it/s]


Unable to remove memmapped file


In [25]:
all_features.shape

(19589, 114)

Exporting Features

In [26]:
all_features.to_csv("allfeatures_5min_120_10", encoding='utf-8', index=False)

In [ ]:
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from joblib import Parallel, delayed
from tqdm.autonotebook import trange
from datetime import timedelta
import multiprocessing

# Step 1: Import necessary modules and libraries
import matplotlib.pyplot as plt
import flirt.reader.empatica
import flirt.with_

# Step 2: Define functions for each step in the pipeline

def import_data(accel_path, reports_path):
    # Importing the datasets based on user-provided paths
    raw_acceleration_data = pd.read_csv(accel_path)
    raw_selfreports_data = pd.read_csv(reports_path)
    
    # Copying the datasets
    df_accel = raw_acceleration_data.copy()
    df_reports = raw_selfreports_data.copy()

    return df_accel, df_reports

def preprocessing(df_reports, df_accel):
    # Cleaning reports
    df_reports = df_reports[df_reports.timeOfEngagement != 0].copy()
    df_reports = df_reports[df_reports.valence != "NONE"].copy()
    df_reports = df_reports[df_reports.arousal != "NONE"].copy()
    df_reports = df_reports[df_reports.context != "NONE"].copy()

    # Renaming the column in df_accel
    df_accel.rename(columns={'timestamp': 'timeOfNotification'}, inplace=True)

    # Converting timeOfNotification from Unix to DateTime
    df_accel["timeOfNotification"] = pd.to_datetime(df_accel["timeOfNotification"], unit="ms")
    df_reports["timeOfNotification"] = pd.to_datetime(df_reports["timeOfNotification"], unit="ms")

    return df_reports, df_accel

def extract_accel_data(row, accel_data, time_window):
    # Convert the time window from minutes to milliseconds
    time_delta = pd.Timedelta(minutes=time_window)
    
    # Calculate the start and end time for the time window
    start_time = row['timeOfNotification'] - time_delta
    end_time = row['timeOfNotification'] + time_delta
    participant_id = row['participantId']

    # Filter accelerometer data for the given participant and time window
    mask = (
        (accel_data['participantId'] == participant_id) &
        (accel_data['timeOfNotification'] >= start_time) &
        (accel_data['timeOfNotification'] <= end_time)
    )
    
    return accel_data[mask]

# Function to create combined dataframe
def create_combined_dataframe(df_reports, df_accel):
    combined_data = []

    for _, row in df_reports.iterrows():
        accel_data = row['accel_data']
        for _, accel_row in accel_data.iterrows():
            combined_data.append({
                'participantId': row['participantId'],
                'selfreport_time': row['timeOfNotification'],
                'valence': row['valence'],
                'arousal': row['arousal'],
                'context': row['context'],
                'accel_time': accel_row['timeOfNotification'],
                'x': accel_row['x'],
                'y': accel_row['y'],
                'z': accel_row['z']
            })

    combined_df = pd.DataFrame(combined_data)
    return combined_df

# Function to import combined dataset
def import_combined_dataset():
    data = pd.read_csv("data_5min_ms.csv")
    return data

# Function to label with report ID
def create_reportid(data):
    data['reportId'] = data.groupby(['participantId', 'selfreport_time']).ngroup() + 1
    col = data.pop("reportId")
    data.insert(0, col.name, col)
    return data

# Function to scale x, y, z data
def scale_xyz_data(data):
    columns_to_scale = ['x', 'y', 'z']
    scaler = StandardScaler()
    scaled_columns = scaler.fit_transform(data[columns_to_scale])
    scaled_df = pd.DataFrame(scaled_columns, columns=columns_to_scale, index=data.index)
    data[columns_to_scale] = scaled_df
    return data

# Function to preprocess data
def preprocess_data(data):
    data = data.drop('participantId', axis=1)
    data = data.drop('selfreport_time', axis=1)
    data.rename(columns={"accel_time": "datetime"}, inplace=True)
    data['datetime'] = pd.DatetimeIndex(data["datetime"]).astype(np.int64)/100000
    return data

# Function to extract features
def extract_features(data):
    features_list = []

    for report_id in data['reportId'].unique():
        temp = data[data['reportId'] == report_id]
        temp_ex = temp[['datetime',"x","y","z"]].copy()
        features = flirt.get_acc_features(temp_ex, window_length=120, window_step_size=10, data_frequency=25)
        features['reportId'] = report_id
        features['arousal'] = temp['arousal'].iloc[0]
        features['valence'] = temp['valence'].iloc[0]
        features['context'] = temp['context'].iloc[0]
        features_list.append(features)

    all_features = pd.concat(features_list)
    return all_features

# Step 3: Create the pipeline
pipeline = Pipeline([
    ('import_datasets', import_data),
    ('preprocessing', preprocessing),
    ('extract_accel_data', extract_accel_data),
    ('create_combined_dataframe', create_combined_dataframe),
    ('import_combined_dataset', import_combined_dataset),
    ('label_with_report_id', create_reportid),
    ('scale_xyz_data', scale_xyz_data),
    ('preprocess_data', preprocess_data),
    ('extract_features', extract_features)
])

# Step 4: Run the pipeline
all_features = pipeline.fit_transform()